In [1]:
import os
import ogr
import shapefile as shp
from math import ceil
# from collections import Counter
from shapely.geometry import Point
from shapely.geometry import Polygon
from rtree import index
import pandas as pd
from datetime import datetime

startTime = datetime.now() #0:00:42.883936

In [2]:
#user inputs
# outPath = "/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/" #folder to dump results
# pointfeat = '/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/ChinaVIIRS.shp' #starting points
outPath = "/Users/itclunie/Desktop/ECON/find industry/speedTest/" #folder to dump results
pointfeat = '/Users/itclunie/Desktop/ECON/find industry/speedTest/VIIRSclustRd.shp' #starting points

polyfeat = outPath + 'outGrid2.shp' #name your output grid

gridCutoff = 10 #gridcell has to have n num of months with values to be flagged
gridHeight = .015 #.015 = 1.5km
gridWidth = .015

In [3]:
def makeGrid(outputGridfn,xmin,xmax,ymin,ymax,gridHeight,gridWidth):
    # convert sys.argv to float
    xmin = float(xmin)
    xmax = float(xmax)
    ymin = float(ymin)
    ymax = float(ymax)
    gridWidth = float(gridWidth)
    gridHeight = float(gridHeight)

    # get rows
    rows = ceil((ymax-ymin)/gridHeight)
    # get columns
    cols = ceil((xmax-xmin)/gridWidth)

    # start grid cell envelope
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin + gridWidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax-gridHeight

    # create output file
    outDriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputGridfn):
        os.remove(outputGridfn)
        
    outDataSource = outDriver.CreateDataSource(outputGridfn)
    outLayer = outDataSource.CreateLayer(outputGridfn,geom_type=ogr.wkbPolygon )
    featureDefn = outLayer.GetLayerDefn()

    # create grid cells
    countcols = 0
    while countcols < cols:
        countcols += 1

        # reset envelope for rows
        ringYtop = ringYtopOrigin
        ringYbottom =ringYbottomOrigin
        countrows = 0

        while countrows < rows:
            countrows += 1
            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            # add new geom to layer
            outFeature = ogr.Feature(featureDefn)
            outFeature.SetGeometry(poly)
            outLayer.CreateFeature(outFeature)
            outFeature.Destroy

            # new envelope for next poly
            ringYtop = ringYtop - gridHeight
            ringYbottom = ringYbottom - gridHeight

        # new envelope for next poly
        ringXleftOrigin = ringXleftOrigin + gridWidth
        ringXrightOrigin = ringXrightOrigin + gridWidth

    # Close DataSources
    outDataSource.Destroy()

In [4]:
#make grid
pointfeatExtent = shp.Reader(pointfeat)

xmin = pointfeatExtent.bbox[0]
xmax = pointfeatExtent.bbox[2]
ymin = pointfeatExtent.bbox[1]
ymax = pointfeatExtent.bbox[3]

makeGrid(polyfeat,xmin,xmax,ymin,ymax,gridHeight,gridWidth)

In [5]:
#A Load the shapefile of polygons and convert it to shapely polygon objects
polygons_sf = shp.Reader(polyfeat)
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes ]
polygons = [Polygon(q) for q in polygon_points]
poly_records = polygons_sf.records()

In [6]:
#B Load the shapefile of points and convert it to shapely point objects
points_sf = shp.Reader(pointfeat)
pntRecords = points_sf.shapeRecords()
point_coords = [q.shape.points[0] for q in pntRecords ]
points = [Point(q.shape.points[0]) for q in pntRecords ]

In [7]:
#C structure tally dictionaries
GRIDdict = {}
subGRIDdict = {}
VIIRSdict = {}
for i in pntRecords:
    VIIRSdict[i.record[4]] = [] #add keys (monthYear) to VIIRSdict & GRIDdict
    GRIDdict[i.record[4]] = []
    
# for i in pntRecords: #fill VIIRSdict.  
#     VIIRSdict[i.record[4]].append(i) #key: monthYear, value: shape and attributes
[VIIRSdict[i.record[4]].append(i) for i in pntRecords]

for i in poly_records: #fill subGRIDdict.  
    subGRIDdict[i[0]] = None ##key= GRIDID, value= {GRIDid:0...    looks like {0:0, 1:0, 2:0 ...

for key in GRIDdict:  #fill GRIDdict. 
    GRIDdict[key] = dict(subGRIDdict)  #key= 8_2012  value=  copies of subGRIDdict  
    

In [8]:
#D Build a spatial index based on the bounding boxes of the polygons
idx = index.Index()

[ idx.insert(i, polygon_shapes[i].bbox) for i in range(len(polygon_shapes)) ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [9]:
#actual points in grid cells matching

countr=0
for key in VIIRSdict:
    tallyHO = []
    point_coords = [ q.shape.points[0] for q in VIIRSdict[key] ]
    points = [ Point(q.shape.points[0]) for q in VIIRSdict[key] ]    
    
    for i in range(len(VIIRSdict[key])): #Iterate through each point

        #Iterate only through the bounding boxes which contain the point 
        
        [tallyHO.append(poly_records[j][0]) for j in idx.intersection(point_coords[i]) if points[i].within(polygons[j])]
        
#         for j in idx.intersection(point_coords[i]):
            
#             #Verify that point is within the polygon itself not just the bounding box
#             if points[i].within(polygons[j]):       
#                 tallyHO.append(poly_records[j][0]) 
#                 break 
                
    resultDict = dict([ (i,tallyHO.count(i)) for i in set(tallyHO) ])

    for rkey in resultDict:
        GRIDdict[key][rkey] = resultDict[rkey] 
            
    countr += 1
    print("I", countr, len( VIIRSdict.keys() ) )

I 1 72
I 2 72
I 3 72
I 4 72
I 5 72
I 6 72
I 7 72
I 8 72
I 9 72
I 10 72
I 11 72
I 12 72
I 13 72
I 14 72
I 15 72
I 16 72
I 17 72
I 18 72
I 19 72
I 20 72
I 21 72
I 22 72
I 23 72
I 24 72
I 25 72
I 26 72
I 27 72
I 28 72
I 29 72
I 30 72
I 31 72
I 32 72
I 33 72
I 34 72
I 35 72
I 36 72
I 37 72
I 38 72
I 39 72
I 40 72
I 41 72
I 42 72
I 43 72
I 44 72
I 45 72
I 46 72
I 47 72
I 48 72
I 49 72
I 50 72
I 51 72
I 52 72
I 53 72
I 54 72
I 55 72
I 56 72
I 57 72
I 58 72
I 59 72
I 60 72
I 61 72
I 62 72
I 63 72
I 64 72
I 65 72
I 66 72
I 67 72
I 68 72
I 69 72
I 70 72
I 71 72
I 72 72


In [10]:
#remove empty rows (ie empty grid squares), pick out hexes of interest, add centroids
dfClean = pd.DataFrame.from_dict(GRIDdict, orient='columns', dtype=None)
dfClean['X'] = None
dfClean['Y'] = None

dropLst = []
for i in range(len(dfClean)):
    row = dfClean.iloc[i]    
    
    if row.count() >= gridCutoff:  #user input
        centroid = polygons[i].centroid
        cX = centroid.coords[0][0]
        cY = centroid.coords[0][1]
        dfClean.at[i,'X'] = cX
        dfClean.at[i,'Y'] = cY
    else:
        dropLst.append(i)

df = dfClean.drop(dropLst)

In [11]:

# keepers2 = []
# for i in range(len(dfClean)):
#     row = dfClean.iloc[i]   
#     rowCount = []
#     [rowCount.append(1) for j in range(len(row)) if "_" in row.index[j] and row[j] > 0]

#     [keepers2.append(i) if sum(rowCount) >= 10]
            
# print(keepers2)

In [12]:


# keepers2 = []
# for i in range(len(dfClean)):
#     row = dfClean.iloc[i] 
    
#     if row.count() >= 10:
#         print(i)
    
#     for j in range(len(row)): #everything except last 3 columns, x y ofinterest
        
#         print(row.index[j], row[j])
#     break

In [13]:
#write output files

df.to_csv(outPath + 'industryFindR_wide.csv')
dfClean['GRIDid'] = dfClean.index
melted = dfClean.melt(id_vars=['GRIDid','X','Y']) 
melted.to_csv(outPath + 'industryFindR_long.csv')

print(datetime.now() - startTime)

0:00:42.883936
